In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
from scipy import stats
import matplotlib.animation as animation

In [2]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [5]:
def inBounds(position):
    return position[0] >= 0 and position[1] >=0 and position[0] < s and position[1] < s

In [6]:
# Block map
s = 100
nPlayers = 100

In [7]:
nInfections = 10
infectionFocus = [[np.random.randint(0,100),np.random.randint(0,100)] for i in range(nInfections)]
infected = []
for elt in infectionFocus:
        for i in range(elt[0]-3,elt[0]+4):
            for j in range(elt[1]-3,elt[1]+4):
                if(abs(i-elt[0])+abs(j-elt[1]) <= 3 and inBounds((i,j))):
                    infected.append((i,j))        
infectedSet = set(infected)
        
        
nWells = 10
wellFocus = [[np.random.randint(0,100),np.random.randint(0,100)] for i in range(nWells)]
welled = []
wellStrength = []
for elt in wellFocus:
        for i in range(elt[0]-6,elt[0]+7):
            for j in range(elt[1]-6,elt[1]+7):
                if(abs(i-elt[0])+abs(j-elt[1]) <= 3):
                    if((i,j) not in welled and inBounds((i,j))):
                        welled.append((i,j))
                        strengthX = stats.poisson(3)
                        strengthY = stats.poisson(3)
                        wellStrength.append([(strengthX.pmf(abs(i-elt[0])))*np.sign(elt[0]-i),
                                             (strengthY.pmf(abs(j-elt[1])))*np.sign(elt[1]-j)])        
wellSet = set(welled)
wellStrength = np.asarray(wellStrength)
#nPatronums = 1
#patronum = [[np.random.randint(0,100),np.random.randint(0,100)] for i in range(nPatronums)]

In [8]:
# Initial parameters
x0A = 0
y0A = 0
x0B = s-1
y0B = s-1

posA = np.asarray([[x0A,y0A]]*nPlayers)
posB = np.asarray([[x0B,y0B]]*nPlayers)
probA = np.asarray([[0.5,0.5]]*nPlayers)
probB = np.asarray([[-0.5,-0.5]]*nPlayers)
infectedA = [0]*nPlayers
infectedB = [0]*nPlayers

In [9]:
def updatePlayers(posA,posB,probA,probB,infectedA,infectedB,infected,infectedSet,welled,wellSet,wellStrength):
    disease = stats.bernoulli(0.1)
    for i in range(nPlayers):
        if tuple(posA[i]) in wellSet:
            probA[i] += wellStrength[welled.index(tuple(posA[i]))]
            
        if tuple(posA[i]) in infectedSet and disease.rvs():
            infectedA[i] = 1
            
        if tuple(posB[i]) in wellSet:
            probB[i] += wellStrength[welled.index(tuple(posB[i]))]
        
        if tuple(posB[i]) in infectedSet and disease.rvs():
            infectedB[i] = 1
            
        xA = stats.norm(0,2*abs(probA[i][0]))
        yA = stats.norm(0,2*abs(probA[i][1]))
        
        position = posA[i] + np.int_(np.asarray([2*abs(xA.rvs())*probA[i][0],2*abs(yA.rvs())*probA[i][1]]))
        
        if inBounds(position):
            posA[i] = position
        
        xB = stats.norm(0,2*abs(probB[i][0]))
        yB = stats.norm(0,2*abs(probB[i][1]))
        
        position2 = posB[i] + np.int_(np.asarray([2*abs(xB.rvs())*probB[i][0],2*abs(yB.rvs())*probB[i][1]]))
        
        if inBounds(position2):
            posB[i] = position2
    
    return posA,posB,probA,probB,infectedA,infectedB

In [10]:
def paint(colors,indices, number=1):
    for elt in indices:
        colors[elt[0],elt[1]] = number
    return colors

In [11]:
def cureInfected(posA,posB,probA,probB,infectedA,infectedB):
    death = stats.bernoulli(0.9)
    indicesA = np.where(infectedA == 1)
    survivalA = np.asarray([death.rvs() for i in range(len(indicesA))])
    indicesA = np.delete(indicesA,np.where(survivalA == 1))
    if(indicesA != []):
        posA = np.delete(posA,indicesA)
        probA = np.delete(probA,indicesA)
        infectedA = np.delete(infectedA,indicesA)
    
    indicesB = np.where(infectedB == 1)
    survivalB = np.asarray([death.rvs() for i in range(len(indicesB))])
    indicesB = np.delete(indicesB,np.where(survivalB == 1))
    if(indicesB != []):
        posB = np.delete(posB,indicesB)
        probB = np.delete(probB,indicesB)
        infectedB = np.delete(infectedB,indicesB)
    
    return posA,posB,probA,probB,infectedA,infectedB

In [12]:
def nextFrame(posA,posB,probA,probB,infectedA,infectedB,infected,infectedSet,welled,wellSet,wellStrength):
    colors = np.zeros((s,s))
    colors = paint(colors,infected,1)
    colors = paint(colors,welled,2)
    posA,posB,probA,probB,infectedA,infectedB = updatePlayers(posA,posB,probA,probB,infectedA,infectedB,infected,infectedSet,welled,wellSet,wellStrength)
    
    posA,posB,probA,probB,infectedA,infectedB = cureInfected(posA,posB,probA,probB,infectedA,infectedB)
    
    colors = paint(colors,posA,-1)
    colors = paint(colors,posB,-2)
    
    return posA,posB,probA,probB,infectedA,infectedB,colors

In [13]:
cmap = mpl.colors.ListedColormap(['blue','red','white','purple','black'])
bounds=[-2.5,-1.5,-0.5,0.5,1.5,2.5]
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

f, ax = plt.subplots(1, 1)
im = ax.imshow(np.zeros((s,s)),interpolation='nearest', cmap = cmap,norm=norm, animated = True)
playing = True

while playing:
    posA,posB,probA,probB,infectedA,infectedB,colors = nextFrame(posA,posB,probA,probB,infectedA,infectedB,infected,infectedSet,welled,wellSet,wellStrength)
    
    im.set_data(colors)
    f.canvas.draw()
    plt.pause(0.001)

/Users/jordiarnau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  """
/Users/jordiarnau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  
/Users/jordiarnau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  del sys.path[0]
/Users/jordiarnau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  


KeyboardInterrupt: 

In [ ]:
elt[0],elt[1]

In [ ]:
posB